# Рекомендательные системы на алгоритмах матричной факторизации

## Загрузка инструментов, данных, контроль воспроизводимости

In [6]:
# !pip install replay-rec rs_datasets implicit -q

In [58]:
import random
import warnings
warnings.simplefilter(action='ignore')

from implicit.bpr import BayesianPersonalizedRanking
from implicit.als import AlternatingLeastSquares
import numpy as np
import pandas as pd
from replay.metrics import HitRate, NDCG, Coverage, Experiment
from replay.preprocessing.converter import CSRConverter
from replay.preprocessing.filters import MinCountFilter, LowRatingFilter
from replay.splitters import TimeSplitter
from rs_datasets import MovieLens

In [7]:
random.seed(0)
np.random.seed(0)

SEED = 0

In [8]:
USER_COL = 'user_id'
ITEM_COL = 'item_id'
RATING_COL = 'rating'
TAMESTAMP = 'timestamp'

In [9]:
data = MovieLens("1m")
ratings = data.ratings
ratings.head()

5.93MB [00:02, 2.70MB/s]                            


,user_id,item_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


## Предобработка данных

In [10]:
ratings = MinCountFilter(
    num_entries=20, groupby_column=USER_COL
    ).transform(ratings)
ratings = MinCountFilter(
    num_entries=20, groupby_column=ITEM_COL
    ).transform(ratings)

print(f'Количество уникальных пользователей: {ratings[USER_COL].nunique()}')
print(f'Количество уникальных фильмов: {ratings[ITEM_COL].nunique()}')

Количество уникальных пользователей: 6040
Количество уникальных фильмов: 3043


### train-test split

Разделим данные по времени так, чтобы 20% взаимодействий оказалось в тестовой выборке. Исключим из тестовой выборки холодных пользователей и холодные объекты.

In [60]:
train, test = TimeSplitter(
    time_threshold=0.2,
    query_column=USER_COL,
    item_column=ITEM_COL,
    drop_cold_users=True,
    drop_cold_items=True
).split(ratings)

train.shape, test.shape

((796392, 4), (103764, 4))

Изменим индексы пользователей и фильмов так, чтобы в нумерации не было пропусков.

In [61]:
train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

In [62]:
all_users = train[USER_COL].astype('category').cat.codes
all_items = train[ITEM_COL].astype('category').cat.codes

user_id2idx = dict(zip(train[USER_COL], all_users))
item_id2idx = dict(zip(train[ITEM_COL], all_items))

train[USER_COL] = train[USER_COL].map(user_id2idx)
train[ITEM_COL] = train[ITEM_COL].map(item_id2idx)
test[USER_COL] = test[USER_COL].map(user_id2idx)
test[ITEM_COL] = test[ITEM_COL].map(item_id2idx)

Будем оценивать качество только на фильмах, имеющих высокие рейтинги в тестовых данных. Для этого оставим рейтинги не ниже 4.

In [63]:
test = LowRatingFilter(value=4, rating_column=RATING_COL).transform(test)
test.shape

(55988, 4)

## Моделирование

Базовый класс для формирования разреженной матрицы рейтингов и получения предсказаний.

In [46]:
class BaseFactorizationModel:
    def __init__(
        self,
        user_col=USER_COL,
        item_col=ITEM_COL,
        reting_col=RATING_COL
    ):
        self.user_col = user_col
        self.item_col = item_col
        self.reting_col = reting_col
        
    def get_rating_matrix(self, data):
        return CSRConverter(
            first_dim_column=self.user_col,
            second_dim_column=self.item_col,
            data_column=self.reting_col
        ).transform(data)
        
    def predict(self, scores, rating_matrix=None, filter_seen=True, k=10):
        if filter_seen:
            scores += np.nan_to_num(rating_matrix.todense() * -1e9)
            
        ind_part = np.argpartition(scores, -k)[:, -k:].copy()
        scores_not_sorted = np.take_along_axis(scores, ind_part, axis=1)
        ind_sorted = np.argsort(scores_not_sorted, axis=1)
        scores_sorted = np.sort(scores_not_sorted, axis=1)
        indices = np.take_along_axis(ind_part, ind_sorted, axis=1)
        
        preds = pd.DataFrame({
            self.user_col: range(scores.shape[0]),
            self.item_col: np.flip(indices, axis=1).tolist(),
            self.reting_col: np.flip(scores_sorted, axis=1).tolist()
        }).explode([self.item_col, self.reting_col])
        
        return preds

### iALS

Сформируем разреженную матрицу рейтингов.

In [64]:
base_model = BaseFactorizationModel()
rating_matrix = base_model.get_rating_matrix(train)

Обучим модель.

In [100]:
ials_model = AlternatingLeastSquares(
    factors=20,
    regularization=1,
    iterations=50,
    use_gpu=False
    )

ials_model.fit((rating_matrix).astype('double'))

  0%|          | 0/50 [00:00<?, ?it/s]

Получим обученные векторы пользователей и объектов и матрицу скоров для каждой пары пользователь-объект.

In [101]:
user_vecs = ials_model.user_factors
item_vecs = ials_model.item_factors
scores = user_vecs.dot(item_vecs.T)

Получим предсказания.

In [102]:
preds_ials = base_model.predict(scores, rating_matrix)

preds_ials.head(10)

,user_id,item_id,rating
0,0,886,0.894744
0,0,900,0.885274
0,0,954,0.770185
0,0,1175,0.761782
0,0,2701,0.747324
0,0,2237,0.729899
0,0,1045,0.691656
0,0,2027,0.671937
0,0,2835,0.659954
0,0,463,0.659621


Посмотрим на статистику предсказанных рейтингов.

In [103]:
scores = scores[scores > -1e+308]
np.min(scores), np.median(scores), np.max(scores)

(-5000000000.0, 0.014671468, 1.5288677)

### BPRMF

Обучим модель

In [104]:
bprmf_model = BayesianPersonalizedRanking(
    factors=20,
    regularization=1,
    iterations=50,
    use_gpu=False,
    random_state=SEED
    )

bprmf_model.fit((rating_matrix).astype('double'))

  0%|          | 0/50 [00:00<?, ?it/s]

Получим обученные векторы пользователей и объектов и матрицу скоров для каждой пары пользователь-объект.

In [105]:
user_vecs = bprmf_model.user_factors
item_vecs = bprmf_model.item_factors
bprmf_scores = user_vecs.dot(item_vecs.T)

Получим предсказания.

In [106]:
preds_bprmf = base_model.predict(bprmf_scores, rating_matrix)

preds_bprmf.head(10)

,user_id,item_id,rating
0,0,954,0.208329
0,0,2189,0.177305
0,0,2237,0.156671
0,0,642,0.15281
0,0,500,0.151411
0,0,943,0.149535
0,0,884,0.14104
0,0,262,0.138908
0,0,1511,0.138891
0,0,1211,0.134756


Посмотрим на статистику предсказанных рейтингов.

In [107]:
bprmf_scores = bprmf_scores[bprmf_scores > -1e+308]
np.min(bprmf_scores), np.median(bprmf_scores), np.max(bprmf_scores)

(-5000000000.0, -0.049011808, 0.208329)

## Metrics

Вычислим метрики качества для top-10 предсказанных фильмов у каждого пользователя.

In [108]:
K = [10]
metrics = Experiment(
    [
        NDCG(K),
        HitRate(K),
        Coverage(K)
    ],
    test,
    train,
    query_column=USER_COL,
    item_column=ITEM_COL,
    rating_column=RATING_COL
)

metrics.add_result("iALS", preds_ials)
metrics.add_result("BPRMF", preds_bprmf)
metrics.results

,NDCG@10,HitRate@10,Coverage@10
iALS,0.210822,0.650893,0.341993
BPRMF,0.186432,0.604464,0.036172
